In [ ]:

from HPS_regular import RegularHeatPumpStudy
from HPS_multistage_condenser import InternalCondenserHeatPumpStudy
from HeatPumpStudy import HeatPumpStudy
from read_csv import read_energy_mix_csv, read_hdd_csv
import pandas as pd
%load_ext autoreload
%autoreload 2

# Assumptions:
- no water heating
- no cooling
- temperature over the whole month is constant
- heat consumption is constant (heat pump is running all the time)
- no losses due to variable speed
- no limit to how slow the heat pump can run

In [51]:
working_fluid="R290" # Propane

# Monthly energy mix in Germany in 2022
# Source: https://www.energy-charts.info/charts/energy/chart.htm?l=de&c=DE&chartColumnSorting=default&source=total&stacking=stacked_percent&partsum=1&interval=month&month=-1&year=2022
file_path = "source_data/energy_mix_2022.csv"
monthly_energy_mix = read_energy_mix_csv(file_path)

# Monthly Heating degree days in germany (in Celsius)
# Source: https://ec.europa.eu/eurostat/databrowser/view/NRG_CHDD_M__custom_6030336/default/table?lang=en
file_path = 'source_data/monthly_hdd_18-22.csv'
monthly_hdd = read_hdd_csv(file_path)

# Monthly average ambient temperatures in Germany (in Celsius)
# Source: https://www.timeanddate.com/weather/germany/berlin/climate
monthly_ambient_temps = [1, 2, 5, 9, 14, 17, 19, 18, 14, 10, 5, 2]

# CO2 emissions per kWh for different energy sources (in kg CO2/kWh_el)
# Assumptions:
coal_co2_emissions = 0.97 #   Source: https://www.volker-quaschning.de/datserv/CO2-spez/index_e.php
natural_gas_co2_emissions = 0.43 #   Source: https://www.volker-quaschning.de/datserv/CO2-spez/index_e.php
nuclear_co2_emissions = 0.012 #   Source: https://www.ipcc.ch/site/assets/uploads/2018/02/ipcc_wg3_ar5_annex-iii.pdf (page 1335)
renewable_co2_emissions = 0.04 #   Source: https://www.ipcc.ch/site/assets/uploads/2018/02/ipcc_wg3_ar5_annex-iii.pdf (page 133)

# TODO:improvement replace assumptions about heating requirements and different insulation classes with more granular data from https://www.ise.fraunhofer.de/content/dam/ise/de/downloads/pdf/Forschungsprojekte/BMWi-03ET1272A-WPsmart_im_Bestand-Schlussbericht.pdf

# Heating demand for new, well-insulated homes (in kWh/m²/year) and assuming 200m² living space per heat pump
# Source: https://www.thermondo.de/info/rat/heizen/heizwaermebedarf-ermitteln/
new_home_heating_demand = 30 * 200

# Heating demand for existing homes with poor insulation (in kWh/m²/year) and assuming 200m² living space per heat pump
# Source: https://www.thermondo.de/info/rat/heizen/heizwaermebedarf-ermitteln/
renovation_heating_demand = 160 * 200

# Heating water temperature for new homes with large radiators (in Celsius)
new_home_heating_temp =  40

# Heating water temperature for renovation with small radiators (in Celsius)
renovation_heating_temp = 65

# Base temperature for calculating HDD (in Celsius)
base_temperature = 18

# ratio of renovation to new homes in 2021
# Source: https://www.waermepumpe.de/fileadmin/user_upload/waermepumpe/08_Sonstige/Filedump/BWP_Branchenstudie_2023_DRUCK.pdf
total_sales_renovations = 100000
total_sales_new_homes = 55000
renovation_to_new_homes_ratio = total_sales_renovations / total_sales_new_homes

In [52]:
# calculate heat demand in kWh for a given number of heating degree days
def monthly_heat_demand_from_hdd(hdd, annual_heat_demand):
    return (hdd / sum(monthly_hdd)) * annual_heat_demand

def calculate_monthly_cop(study:HeatPumpStudy, heating_temp, avg_ambient_temp, monthly_heat_demand):
    # convert monthly heat demand from kWh to W assuming 100% duty cycle
    study.Q_out = monthly_heat_demand / (24 * 30)*1000
    study.setup_network(iterinfo=False)
    heating_temp = max(avg_ambient_temp+20, heating_temp) # heating temp is at least 20 degrees above ambient temp
    if isinstance(study, InternalCondenserHeatPumpStudy):
        study.set_boundary_conditions(T_cond=heating_temp+5, T_evap=avg_ambient_temp-5, T_consumer=heating_temp)
    else:
        study.set_boundary_conditions(T_cond=heating_temp+5, T_evap=avg_ambient_temp-5)
    study.solve()
    return study.calculate_cop()

def calculate_monthly_co2_emissions(energy_mix, cop, heat_demand_kwh):
    co2_emissions_per_kwh = (
        energy_mix["coal"] * coal_co2_emissions
        + energy_mix["natural_gas"] * natural_gas_co2_emissions
        + energy_mix["nuclear"] * nuclear_co2_emissions
        + energy_mix["renewable"] * renewable_co2_emissions
    )
    return (heat_demand_kwh / cop) * co2_emissions_per_kwh

def calculate_monthly_energy_consumption(cop, heat_demand_kwh):
    return heat_demand_kwh / cop

def calculate_annual_co2_emissions(heatpump:HeatPumpStudy, annual_heating_demand, heating_temp):
    annual_co2_emissions = 0
    for i in range(12):
        monthly_heat_demand = monthly_heat_demand_from_hdd(monthly_hdd[i], annual_heating_demand)
        avg_ambient_temp = monthly_ambient_temps[i]
        cop = calculate_monthly_cop(heatpump,heating_temp, avg_ambient_temp, monthly_heat_demand)
        co2_emissions = calculate_monthly_co2_emissions(monthly_energy_mix[i], cop, monthly_heat_demand)
        #print(f"CO2 emissions for month {str(i+1)}: {str(co2_emissions)} kg")
        annual_co2_emissions += co2_emissions
    return annual_co2_emissions

def calculate_annual_energy_consumption(heatpump:HeatPumpStudy, annual_heating_demand, heating_temp):
    annual_energy_consumption = 0
    for i in range(12):
        monthly_heat_demand = monthly_heat_demand_from_hdd(monthly_hdd[i], annual_heating_demand)
        avg_ambient_temp = monthly_ambient_temps[i]
        cop = calculate_monthly_cop(heatpump,heating_temp, avg_ambient_temp, monthly_heat_demand)
        energy_consumption = calculate_monthly_energy_consumption(cop, monthly_heat_demand)
        #print(f"Energy consumption for month {str(i+1)}: {str(energy_consumption)} kWh")
        annual_energy_consumption += energy_consumption
    return annual_energy_consumption

In [53]:
def main():
    #Sales prognosis for 2026, 2027
    total_sales = [3600, 12700]
    # yearly growth of 30% beyond that
    yearly_growth = 0.3
    total_sales.extend([round(total_sales[-1] * (1 + yearly_growth) ** i) for i in range(1, 10)])
    years=len(total_sales)
    print(total_sales)

    new_home_heatpumps = [round(total_sales[i] / (1 + renovation_to_new_homes_ratio)) for i in range(years)]
    renovation_heatpumps = [round(total_sales[i] /(1+1/ renovation_to_new_homes_ratio)) for i in range(years)]

    regular_heatpump=RegularHeatPumpStudy()
    regular_heatpump_new_home_annual_energy = calculate_annual_energy_consumption(regular_heatpump, new_home_heating_demand, new_home_heating_temp)
    regular_heatpump_new_home_annual_co2 = calculate_annual_co2_emissions(regular_heatpump, new_home_heating_demand, new_home_heating_temp)
    regular_heatpump_renovation_annual_energy = calculate_annual_energy_consumption(regular_heatpump, renovation_heating_demand, renovation_heating_temp)
    regular_heatpump_renovation_annual_co2 = calculate_annual_co2_emissions(regular_heatpump, renovation_heating_demand, renovation_heating_temp)
    
    improved_heatpump=InternalCondenserHeatPumpStudy(expansion_device="expander", N=1)
    improved_heatpump_new_home_annual_energy = calculate_annual_energy_consumption(improved_heatpump, new_home_heating_demand, new_home_heating_temp)
    improved_heatpump_new_home_annual_co2 = calculate_annual_co2_emissions(improved_heatpump, new_home_heating_demand, new_home_heating_temp)
    improved_heatpump_renovation_annual_energy = calculate_annual_energy_consumption(improved_heatpump, renovation_heating_demand, renovation_heating_temp)
    improved_heatpump_renovation_annual_co2 = calculate_annual_co2_emissions(improved_heatpump, renovation_heating_demand, renovation_heating_temp)


    print(f"relative energy savings in new homes: {round((1-improved_heatpump_new_home_annual_energy/regular_heatpump_new_home_annual_energy)*100,2)}%")
    print(f"relative co2 savings in new homes: {round((1-improved_heatpump_new_home_annual_co2/regular_heatpump_new_home_annual_co2)*100,2)}%")
    print(f"relative energy savings in renovations: {round((1-improved_heatpump_renovation_annual_energy/regular_heatpump_renovation_annual_energy)*100,2)}%")
    print(f"relative co2 savings in renovations: {round((1-improved_heatpump_renovation_annual_co2/regular_heatpump_renovation_annual_co2)*100,2)}%")

    print(f"absoulte energy savings in new homes: {round(regular_heatpump_new_home_annual_energy-improved_heatpump_new_home_annual_energy,2)} kWh")
    print(f"absoulte co2 savings in new homes: {round(regular_heatpump_new_home_annual_co2-improved_heatpump_new_home_annual_co2,2)} kg")
    print(f"absoulte energy savings in renovations: {round(regular_heatpump_renovation_annual_energy-improved_heatpump_renovation_annual_energy,2)} kWh")
    print(f"absoulte co2 savings in renovations: {round(regular_heatpump_renovation_annual_co2-improved_heatpump_renovation_annual_co2,2)} kg")
    
    results = [
        [
            year,
            new_home_heatpumps[i],
            renovation_heatpumps[i],
            regular_heatpump_new_home_annual_energy * new_home_heatpumps[i] + regular_heatpump_renovation_annual_energy * renovation_heatpumps[i],
            regular_heatpump_new_home_annual_co2 * new_home_heatpumps[i] + regular_heatpump_renovation_annual_co2 * renovation_heatpumps[i],
            improved_heatpump_new_home_annual_energy * new_home_heatpumps[i] + improved_heatpump_renovation_annual_energy * renovation_heatpumps[i],
            improved_heatpump_new_home_annual_co2 * new_home_heatpumps[i] + improved_heatpump_renovation_annual_co2 * renovation_heatpumps[i],
            ((regular_heatpump_new_home_annual_energy - improved_heatpump_new_home_annual_energy) * new_home_heatpumps[i] + (regular_heatpump_renovation_annual_energy - improved_heatpump_renovation_annual_energy) * renovation_heatpumps[i])/1e6,
            ((regular_heatpump_new_home_annual_co2 - improved_heatpump_new_home_annual_co2) * new_home_heatpumps[i] + (regular_heatpump_renovation_annual_co2 - improved_heatpump_renovation_annual_co2) * renovation_heatpumps[i])/1e3,
        ]
        for year, i in enumerate(range(years), start=2026)
    ]
    
    # we want to calculate the cumulative savings saved that year.
    # since the installed heat pumps continue to provide savings in the coming years, 
    # we need to sum up the savings from the previous years
    results_cumulative=results.copy()
    for i in range(1, len(results)):
        results_cumulative[i][3] = sum([results[j][3] for j in range(i + 1)])
        results_cumulative[i][4] = sum([results[j][4] for j in range(i + 1)])
        results_cumulative[i][5] = sum([results[j][5] for j in range(i + 1)])
        results_cumulative[i][6] = sum([results[j][6] for j in range(i + 1)])
        results_cumulative[i][7] = sum([results[j][7] for j in range(i + 1)])
        results_cumulative[i][8] = sum([results[j][8] for j in range(i + 1)])
        
    #store results to excel sheet in ouput/results.xlsx
    results_df = pd.DataFrame(
        results_cumulative,
        columns=[
            "Year",
            "New homes",
            "Renovations",
            "Regular heat pump energy consumption [kWh]]",
            "Regular heat pump CO2 emissions [kg]",
            "Improved heat pump energy consumption",
            "Improved heat pump CO2 emissions",
            "Energy savings [GWh]",
            "CO2 savings [Tons]",
        ],
    )
    results_df.to_excel("output/results.xlsx", index=False)

In [54]:
main()

[3600, 12700, 16510, 21463, 27902, 36272, 47154, 61300, 79691, 103598, 134677]


relative energy savings in new homes: 12.86%
relative co2 savings in new homes: 12.86%
relative energy savings in renovations: 21.74%
relative co2 savings in renovations: 21.75%
absoulte energy savings in new homes: 166.94 kWh
absoulte co2 savings in new homes: 62.99 kg
absoulte energy savings in renovations: 2542.16 kWh
absoulte co2 savings in renovations: 958.9 kg


In [55]:
import numpy as np
# Crankcase heater power rating (W)
crankcase_heater_power = 70

# Assumption: Crankcase heater operates when the ambient temperature is below 10°C
heater_operating_hours_per_day = [6 if temp < 10 else 0 for temp in monthly_ambient_temps]

monthly_hdd = [max(base_temperature - temp, 0) for temp in monthly_ambient_temps]

# Total number of days the crankcase heater operates in a year
operating_days_per_month = np.array(heater_operating_hours_per_day) * np.array(list(monthly_hdd))
total_operating_days = sum(operating_days_per_month)

# Energy wasted on crankcase heaters in a year (kWh)
energy_wasted = crankcase_heater_power * total_operating_days / 1000